In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [6]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192",temperature=0.5)